<a href="https://colab.research.google.com/github/EltonLunardi/IC_Dataset_IDS/blob/main/IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
dados = pd.read_csv("dataset.csv")
pd.set_option('display.max_columns', 500)
max_time = 1000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dados['class2'] = dados['class']
dados['class2'].astype(bool)
dados['class2'].replace({'normal':False,'injection':True,'high_StNum':True,'random_replay':True,'poisoned_high_rate':True,'masquerade_fake_normal':True,'masquerade_fake_fault':True,'inverse_replay':True},inplace=True)

In [ ]:
#dados.drop(['nome_coluna'],axis='columns',inplace=True) #Dropa coluna definida

In [ ]:
#dados.dtypes

In [ ]:
dados.head()

In [ ]:
dados['Results_VP'].value_counts()

In [ ]:
dados['Results_VN'].value_counts()

In [ ]:
dados['Results_FP'].value_counts()

In [ ]:
dados['Results_FN'].value_counts()

In [ ]:
dados.drop(['R6.0_FP'],axis='columns',inplace=True) #Dropa coluna definida

In [ ]:
#R1 GOOSE messages must have MAC address starting with 01-0c-cd-01
dados['teste_ethSrc'] = dados.ethSrc.str.contains("01:0c:cd:01")
dados['teste_ethDst']= dados.ethDst.str.contains("01:0c:cd:01")
dados['R1'] = dados[['teste_ethSrc','teste_ethDst']].apply(lambda x: x['teste_ethSrc'] and x['teste_ethDst'],axis='columns')
dados['R1_VN'] = (dados['R1'] == dados['class2']) & (dados['class2'])
dados['R1_VP'] = (dados['R1'] == dados['class2']) & (dados['class2'] == False)
dados['R1_FN'] = (dados['R1'] != dados['class2']) & (dados['class2'] == False)
dados['R1_FP'] = (dados['R1'] != dados['class2']) & (dados['class2'])

In [ ]:
#R2 GOOSE messages must have the TPID field with value 0x8100
dados['R2'] = dados.TPID.str.contains("0x8100")
dados['R2_VN'] = (dados['R2'] == dados['class2']) & (dados['class2'])
dados['R2_VP'] = (dados['R2'] == dados['class2']) & (dados['class2'] == False)
dados['R2_FN'] = (dados['R2'] != dados['class2']) & (dados['class2'] == False)
dados['R2_FP'] = (dados['R2'] != dados['class2']) & (dados['class2'])

In [ ]:
#R3 GOOSE messages must have the ethertype field equal to 0x88B8
dados['R3'] = dados.ethType.str.contains("0x88B8")
dados['R3_VN'] = (dados['R3'] == dados['class2']) & (dados['class2'])
dados['R3_VP'] = (dados['R3'] == dados['class2']) & (dados['class2'] == False)
dados['R3_FN'] = (dados['R3'] != dados['class2']) & (dados['class2'] == False)
dados['R3_FP'] = (dados['R3'] != dados['class2']) & (dados['class2'])

In [ ]:
#R4 GOOSE messages must have TimeAllowedToLive equal to double of the value of MaxTime
dados['R4'] = dados['gooseTimeAllowedtoLive'] == (max_time*2)
dados['R4_VN'] = (dados['R4'] == dados['class2']) & (dados['class2'])
dados['R4_VP'] = (dados['R4'] == dados['class2']) & (dados['class2'] == False)
dados['R4_FN'] = (dados['R4'] != dados['class2']) & (dados['class2'] == False)
dados['R4_FP'] = (dados['R4'] != dados['class2']) & (dados['class2'])

In [ ]:
#R5 GOOSE messages must have the APPID field formatted as a 4-byte hexadecimal (e.g, 0000-3FFF)
  # Formatado em hexadecimal ??dados[R5] = hex(dados[APPID])??
dados['R5'] = dados['gooseAppid'].apply(lambda x: x.split('0x')[1][:4]+'-'+ x.split('0x')[1][4:])
dados['R5_VN'] = (dados['R5'] == dados['class2']) & (dados['class2'])
dados['R5_VP'] = (dados['R5'] == dados['class2']) & (dados['class2'] == False)
dados['R5_FN'] = (dados['R5'] != dados['class2']) & (dados['class2'] == False)
dados['R5_FP'] = (dados['R5'] != dados['class2']) & (dados['class2'])

In [ ]:
#R6 Consecutive GOOSE messages must have consistent values for fields:gocbRef, gooseTimeAllowedtoLive, datSet, goID, test,confRev,ndsCom and numDatSetEntries
dados['R6.0'] = dados['gocbRef'] == dados['gocbRef'].shift()
dados['R6.0_VN'] = (dados['R6.0'] == dados['class2']) & (dados['class2'])
dados['R6.0_VP'] = (dados['R6.0'] == dados['class2']) & (dados['class2'] == False)
dados['R6.0_FN'] = (dados['R6.0'] != dados['class2']) & (dados['class2'] == False)
dados['R6.0_FP'] = (dados['R6.0'] != dados['class2']) & (dados['class2'])

dados['R6.1'] = dados['gooseTimeAllowedtoLive'].shift() - dados['gooseTimeAllowedtoLive']
dados.loc[dados['R6.1'] <= 0,'R6.1'] = True 
dados.loc[dados['R6.1'] > 0,'R6.1'] = False
dados['R6.1_VN'] = (dados['R6.1'] == dados['class2']) & (dados['class2'])
dados['R6.1_VP'] = (dados['R6.1'] == dados['class2']) & (dados['class2'] == False)
dados['R6.1_FN'] = (dados['R6.1'] != dados['class2']) & (dados['class2'] == False)
dados['R6.1_FP'] = (dados['R6.1'] != dados['class2']) & (dados['class2'])

dados['R6.2'] = dados['datSet'] == dados['datSet'].shift()
dados['R6.2_VN'] = (dados['R6.2'] == dados['class2']) & (dados['class2'])
dados['R6.2_VP'] = (dados['R6.2'] == dados['class2']) & (dados['class2'] == False)
dados['R6.2_FN'] = (dados['R6.2'] != dados['class2']) & (dados['class2'] == False)
dados['R6.2_FP'] = (dados['R6.2'] != dados['class2']) & (dados['class2'])

dados['R6.3'] = dados['goID'] == dados['goID'].shift()
dados['R6.3_VN'] = (dados['R6.3'] == dados['class2']) & (dados['class2'])
dados['R6.3_VP'] = (dados['R6.3'] == dados['class2']) & (dados['class2'] == False)
dados['R6.3_FN'] = (dados['R6.3'] != dados['class2']) & (dados['class2'] == False)
dados['R6.3_FP'] = (dados['R6.3'] != dados['class2']) & (dados['class2'])

dados['R6.4'] = dados['test'] == dados['test'].shift()
dados['R6.4_VN'] = (dados['R6.4'] == dados['class2']) & (dados['class2'])
dados['R6.4_VP'] = (dados['R6.4'] == dados['class2']) & (dados['class2'] == False)
dados['R6.4_FN'] = (dados['R6.4'] != dados['class2']) & (dados['class2'] == False)
dados['R6.4_FP'] = (dados['R6.4'] != dados['class2']) & (dados['class2'])


dados['R6.5'] = dados['confRev'].shift() - dados['confRev']
dados.loc[dados['R6.5'] <= 0,'R6.5'] = True 
dados.loc[dados['R6.5'] > 0,'R6.5'] = False
dados['R6.5_VN'] = (dados['R6.5'] == dados['class2']) & (dados['class2'])
dados['R6.5_VP'] = (dados['R6.5'] == dados['class2']) & (dados['class2'] == False)
dados['R6.5_FN'] = (dados['R6.5'] != dados['class2']) & (dados['class2'] == False)
dados['R6.5_FP'] = (dados['R6.5'] != dados['class2']) & (dados['class2'])

dados['R6.6'] = dados['ndsCom'] == dados['ndsCom'].shift()
dados['R6.6_VN'] = (dados['R6.6'] == dados['class2']) & (dados['class2'])
dados['R6.6_VP'] = (dados['R6.6'] == dados['class2']) & (dados['class2'] == False)
dados['R6.6_FN'] = (dados['R6.6'] != dados['class2']) & (dados['class2'] == False)
dados['R6.6_FP'] = (dados['R6.6'] != dados['class2']) & (dados['class2'])

dados['R6.7'] = dados['numDatSetEntries'].shift() - dados['numDatSetEntries']
dados.loc[dados['R6.7'] <= 0,'R6.7'] = True 
dados.loc[dados['R6.7'] > 0,'R6.7'] = False 
dados['R6.7_VN'] = (dados['R6.7'] == dados['class2']) & (dados['class2'])
dados['R6.7_VP'] = (dados['R6.7'] == dados['class2']) & (dados['class2'] == False)
dados['R6.7_FN'] = (dados['R6.7'] != dados['class2']) & (dados['class2'] == False)
dados['R6.7_FP'] = (dados['R6.7'] != dados['class2']) & (dados['class2'])

dados['R6.8'] = dados['t'].shift() - dados['t'] #subtrair os dois valores 
dados.loc[dados['R6.8'] <= 0,'R6.8'] = True #substração negativa significa q o de cima é menor q o de baixo
dados.loc[dados['R6.8'] > 0,'R6.8'] = False #substração positiva significa q o de cima é maior q o de baixo
dados['R6.8_VN'] = (dados['R6.8'] == dados['class2']) & (dados['class2'])
dados['R6.8_VP'] = (dados['R6.8'] == dados['class2']) & (dados['class2'] == False)
dados['R6.8_FN'] = (dados['R6.8'] != dados['class2']) & (dados['class2'] == False)
dados['R6.8_FP'] = (dados['R6.8'] != dados['class2']) & (dados['class2'])
  

In [ ]:
#R7 GOOSE messages must have the APPID field matching the last two octets of the destination multicast address
dados['ethDst_formatR7'] = dados['ethDst'].apply(lambda x: "0x"+"".join(x.split(":")[-4:])) #0x no começo indica o formato hex. Join ele une a string q foi separada pelo ":", -4 no começo indica quantas posições vamos considerar
dados['R7'] = dados['ethDst_formatR7'] == dados['gooseAppid']
dados['R7_VN'] = (dados['R7'] == dados['class2']) & (dados['class2'])
dados['R7_VP'] = (dados['R7'] == dados['class2']) & (dados['class2'] == False)
dados['R7_FN'] = (dados['R7'] != dados['class2']) & (dados['class2'] == False)
dados['R7_FP'] = (dados['R7'] != dados['class2']) & (dados['class2'])

In [ ]:
#R8 The IED control block name must be consistent with the value of the goID field (i.e., the 𝐿𝐷/𝐿𝑁 value in the gocoRef field must match the datSet field from the GOOSE APDU)
dados['R8'] = dados.gocbRef.str.contains('datSet')
dados['R8_VN'] = (dados['R8'] == dados['class2']) & (dados['class2'])
dados['R8_VP'] = (dados['R8'] == dados['class2']) & (dados['class2'] == False)
dados['R8_FN'] = (dados['R8'] != dados['class2']) & (dados['class2'] == False)
dados['R8_FP'] = (dados['R8'] != dados['class2']) & (dados['class2'])

In [ ]:
#R9 The size of frames containing GOOSE messages should be equal to 8 𝑏𝑦𝑡𝑒𝑠+ 𝐴𝑃𝐷𝑈 𝑠𝑖𝑧𝑒, and 𝐴𝑃𝐷𝑈 𝑠𝑖𝑧𝑒 should be less than 1492 𝑏𝑦𝑡𝑒𝑠
dados['R9'] = (dados['APDUSize'] >= 8) & (dados['gooseLen'] < 1492)
dados['R9_VN'] = (dados['R9'] == dados['class2']) & (dados['class2'])
dados['R9_VP'] = (dados['R9'] == dados['class2']) & (dados['class2'] == False)
dados['R9_FN'] = (dados['R9'] != dados['class2']) & (dados['class2'] == False)
dados['R9_FP'] = (dados['R9'] != dados['class2']) & (dados['class2'])

In [ ]:
#R10 The SqNum in GOOSE messages should be set to zero whenever the value of the StNum changes (w.r.t the previous message)
dados['R10'] = dados['StNum'].shift() - dados['StNum']
dados['R10'] = ((dados['R10'] == 0)) | ((dados['R10'] == 1) & (dados['SqNum'] == 1))
dados['R10_VN'] = (dados['R10'] == dados['class2']) & (dados['class2'])
dados['R10_VP'] = (dados['R10'] == dados['class2']) & (dados['class2'] == False)
dados['R10_FN'] = (dados['R10'] != dados['class2']) & (dados['class2'] == False)
dados['R10_FP'] = (dados['R10'] != dados['class2']) & (dados['class2'])

In [ ]:
#R11 The number of messages captured in an interval must not exceed a pre-defined threshold (20% above the expected maximum)

In [ ]:
#R12 The number of messages captured in an interval must not be equal to zero
dados['R12'] = dados['GooseTimestamp'] - dados['GooseTimestamp'].shift()
dados['R12'] = ((dados['R12'] >= 0))
dados['R12_VN'] = (dados['R12'] == dados['class2']) & (dados['class2'])
dados['R12_VP'] = (dados['R12'] == dados['class2']) & (dados['class2'] == False)
dados['R12_FN'] = (dados['R12'] != dados['class2']) & (dados['class2'] == False)
dados['R12_FP'] = (dados['R12'] != dados['class2']) & (dados['class2'])

In [ ]:
#R15 The Recency metric, represented by the last GOOSE message’s arrival, must respect a minimum and a maximum threshold
dados['R15'] =((dados['timestampDiff'] >= 0.003) & (dados['timestampDiff'] <= 1000))
dados['R15_VN'] = (dados['R15'] == dados['class2']) & (dados['class2'])
dados['R15_VP'] = (dados['R15'] == dados['class2']) & (dados['class2'] == False)
dados['R15_FN'] = (dados['R15'] != dados['class2']) & (dados['class2'] == False)
dados['R15_FP'] = (dados['R15'] != dados['class2']) & (dados['class2'])

In [ ]:
#R16 The Frequency metric, represented by the average number of received GOOSE messages, must respect a minimum and a maximum predefined threshold
  #Quantidade de linhas(mensagens) em 1s foi   maior que 7 e menor que 1

In [ ]:
#R17 The Monetary metric, represented by the total number of received GOOSE messages, must be within a predefined threshold [46]. The difference from rule #𝑅11) is that this rule considers only received GOOSE messages
  #Igual a R11

In [ ]:
#R18 Only messages with specific source port, IP and MAC addresses are allowed
#srcEth
#comparar com os valores de srcEth e srcDst

In [ ]:
#R19 Only MMS, COTP, TPKT, and SNTP protocols are allowed on the station level network and only the GOOSE, SV, and IEEE 1588 protocols are allowed on the process level network
dados['R19'] = ((dados['protocol'] == 'GOOSE'))
dados['R19_VN'] = (dados['R19'] == dados['class2']) & (dados['class2'])
dados['R19_VP'] = (dados['R19'] == dados['class2']) & (dados['class2'] == False)
dados['R19_FN'] = (dados['R19'] != dados['class2']) & (dados['class2'] == False)
dados['R19_FP'] = (dados['R19'] != dados['class2']) & (dados['class2'])

In [ ]:
#R21 The number of bytes that travel per second must not exceed a predefined threshold

In [ ]:
#R22 The number of packets that travel per second must not exceed a predefined threshold
  #Igual a R11

In [ ]:
#R23 The length of the packet (specified in the packet header) must not exceed a predefined threshold
dados['R23'] = (dados['frameLen'] >= 300)
dados['R23_VN'] = (dados['R23'] == dados['class2']) & (dados['class2'])
dados['R23_VP'] = (dados['R23'] == dados['class2']) & (dados['class2'] == False)
dados['R23_FN'] = (dados['R23'] != dados['class2']) & (dados['class2'] == False)
dados['R23_FP'] = (dados['R23'] != dados['class2']) & (dados['class2'])

In [ ]:
#R24 The total size of the packet must not exceed a predefined threshold
dados['R24'] = (dados['gooseLen'] >= 275)
dados['R24_VN'] = (dados['R24'] == dados['class2']) & (dados['class2'])
dados['R24_VP'] = (dados['R24'] == dados['class2']) & (dados['class2'] == False)
dados['R24_FN'] = (dados['R24'] != dados['class2']) & (dados['class2'] == False)
dados['R24_FP'] = (dados['R24'] != dados['class2']) & (dados['class2'])

In [ ]:
dados['line_result'] = (dados['R1'] | dados['R2'] | dados['R3'] | dados['R4'] | dados['R5'] | dados['R6.0'] | dados['R6.1'] | dados['R6.2'] | dados['R6.3'] | dados['R6.4'] | dados['R6.5'] | dados['R6.6'] | dados['R6.7'] | dados['R6.8'] | dados['R7'] | dados['R8'] | dados['R9'] | dados['R10'] | dados['R12'] | dados['R15'] | dados['R19'] | dados['R23'] | dados['R24'] )

In [ ]:
dados['Results_VN'] = (dados['line_result'] == dados['class2']) & (dados['class2'])
dados['Results_VP'] = (dados['line_result'] == dados['class2']) & (dados['class2'] == False)
dados['Results_FN'] = (dados['line_result'] != dados['class2']) & (dados['class2'] == False)
dados['Results_FP'] = (dados['line_result'] != dados['class2']) & (dados['class2'])

# Impressão de resultados


In [ ]:
dados[['class','R1','R1_VN','R1_VP','R1_FN','R1_FP','R2','R2_VN','R2_VP','R2_FN','R2_FP','R3','R3_VN','R3_VP','R3_FN','R3_FP','R4','R4_VN','R4_VP','R4_FN','R4_FP','R5','R5_VN','R5_VP','R5_FN','R5_FP','R6.0','R6.0_VN','R6.0_VP','R6.0_FN','R6.0_FP','R6.1','R6.1_VN','R6.1_VP','R6.1_FN','R6.1_FP','R6.2','R6.2_VN','R6.2_VP','R6.2_FN','R6.2_FP','R6.3','R6.3_VN','R6.3_VP','R6.3_FN','R6.3_FP','R6.4','R6.4_VN','R6.4_VP','R6.4_FN','R6.4_FP','R6.5','R6.5_VN','R6.5_VP','R6.5_FN','R6.5_FP','R6.6','R6.6_VN','R6.6_VP','R6.6_FN','R6.6_FP','R6.7','R6.7_VN','R6.7_VP','R6.7_FN','R6.7_FP','R6.8','R6.8_VN','R6.8_VP','R6.8_FN','R6.8_FP','R7','R7_VN','R7_VP','R7_FN','R7_FP','R8','R8_VN','R8_VP','R8_FN','R8_FP','R9','R9_VN','R9_VP','R9_FN','R9_FP','R10','R10_VN','R10_VP','R10_FN','R10_FP','R12','R12_VN','R12_VP','R12_FN','R12_FP','R15','R15_VN','R15_VP','R15_FN','R15_FP','R19','R19_VN','R19_VP','R19_FN','R19_FP','R23','R23_VN','R23_VP','R23_FN','R23_FP','R24','R24_VN','R24_VP','R24_FN','R24_FP','line_result','Results_VN','Results_VP','Results_FN','Results_FP']].to_csv('/Resultados.csv',index=False) #Ordenação das colunas a ser feito posteriormente

In [ ]:
dados[['class','R1','R2','R3','R4','R5','R6.0','R6.1','R6.2','R6.3','R6.4','R6.5','R6.6','R6.7','R6.8','R7','R8','R9','R10','R12','R15','R19','R23','R24','line_result']].to_csv('resultados-regras.csv',index=False) 

In [ ]:
'''
Analise do F1 Score
VN = line_result == class == 'true' # true = attack
VP = line_result == class == 'false' # false = normal
FN = line_result != class and line_result == 'true' # true = attack
FP = line_result != class and line_result == 'false' # false = normal
'''